# Possible bugs

2 things are unconvincing:
- formula of UCT with prior: doesn't seem to explore enough; also investigate the effect of the default Q value (e.g. 0 vs parent.value!
- entropy bonus, the flag is turned to true, but the effects are not observed, not even for h=0.1

## UCT formula

**Old code**:
```python
def ucb_score(self, parent, child, eps=1e-3):
    """
    The score for a node is based on its value, plus an exploration bonus.
    """
    exploration_term = self.ucb_c*np.sqrt(np.log(parent.visit_count)/(child.visit_count+eps))

    if child.visit_count > 0:
        # Mean value Q
        value_term = child.reward + self.discount*child.value() 
    else:
        value_term = 0

    return value_term + exploration_term, value_term, exploration_term
```

**Formula from Kocsis-Szepesvári-2006**:
$$
a_t = \arg \underset{a}{\max} \left[Q(s,a) + \sqrt{\frac{ 2\ln N_{s}}{N_{s,a}}}\right]
$$

Where is assumed that all actions have to be explored at least once before starting to consider the Q values (in other words for all actions which have $N_{s,a}=0$ the exploration bonus (or bias term) is infinite and we need to sample one og them randomly.

I can't find any mention of what should be the default value for the Q before they are explored, but given that we first explore all of them once and then start using the formula, the default value will not have influence on the allocation of visit counts.

**What I actually implemented**:
$$
a_t = \arg \underset{a}{\max} \left[Q(s,a) + c \sqrt{\frac{\ln N_{s}}{N_{s,a} + \epsilon}}\right]
$$

where $c=1$, $\epsilon=10^{-3}$. This formula does exactly what it's supposed to, except for a factor $\sqrt{2}$ which is lacking in the exploration term (thus it would explore more).
Q values are defaulted at 0 when the action still has to be explored, but for an action space of at most 5 and the selected value of $\epsilon$, the UCB value of unexplored actions will be:
- 0 at the beginning (but it's the same for all actions)
- 26.32768848 after the first action has been selected
- 33.14532077 after the second action has been selected
- 37.23297411 after the third action has been selected
- 40.11780044 after the fourth action has been selected

Since these values are all much bigger than 1 (maximum Q value), the default initialization of the Qs does not change the exploration process.


## p-UCT formulas

**Old code**:

```python
def ucb_score(self, parent, child):
    """
    The score for a node is based on its value, plus an exploration bonus.
    """
    exploration_term = self.ucb_c*child.prior*np.sqrt(np.log(parent.visit_count)/(child.visit_count+1))

    if child.visit_count > 0:
        # Mean value Q
        value_term = child.reward + self.discount*child.value() 
    else:
        value_term = 0

    return value_term + exploration_term, value_term, exploration_term
```

**Formula from Rosin-2012** (reference from AlphaGo when they talk about their own exploration bonus):
$$
a_t = \arg \underset{a}{\max} \left[Q(s,a) + c(N_s, N_{s,a}) - m(N_s, a) \right]
$$
where:
$$
c(N_s, N_{s,a}) =
\begin{cases}
\sqrt{\frac{3 \log N_s}{2 N_{s,a}}} & \text{if}~ N_{s,a} > 0 \\
0 & \text{otherwise}
\end{cases}
$$

$$
m(N_{s}, a) = 
\begin{cases}
\frac{2}{M(s,a)}\sqrt{\frac{\ln N_s}{N_s}} & \text{if}~ N_s > 1 \\
\frac{2}{M(s,a)} & \text{otherwise}
\end{cases}
$$

Where $M(s,a) = \frac{\sqrt{P(s,a)}}{\sum_b P(s,b)}$ (Sec. 3.2 of the paper).

Also $Q(s,a)$ is defaulted to 1 (max possible payoff) if $N_{s,a}=0$.

**Formula from AlphaGo:**
$$
a_t = \arg \underset{a}{\max} \left[Q(s,a) + c P(s,a) \frac{\sqrt{\sum_b N_{s,b}}}{1 + N_{s,a}} \right]
$$

I still have to understand at which value $Q(s,a)$ is defaulted if $N_{s,a}=0$.

**What I actually implemented:**
$$
a_t = \arg \underset{a}{\max} \left[Q(s,a) + c P(s,a) \sqrt\frac{{\ln \sum_b N_{s,b}}}{1 + N_{s,a}} \right]
$$

Also $Q(s,a)$ is defaulted to 0 (payoffs are in interval [-1,1]) if $N_{s,a}=0$.

## Theoretical analysis of Rosin's p-UCT formula

Assuming 2 actions with prior $p$ and $q=1-p$, unknown deterministic payoffs $Q_p$ and $Q_q$ $\in [-1,1]$ (this is a simplifying assumption, in reality we will have a distribution of $Q_p$ to average upon) and $p>q$, after how many "selection rounds" $n$ (this would be the $N_s$, i.e. the parent visit count) will action $q$ be chosen?

First we compute the weights $M_p$ and $M_q$:
$$
M_p = \frac{\sqrt{p}}{\sqrt{p}+\sqrt{q}} \\
M_q = \frac{\sqrt{q}}{\sqrt{p}+\sqrt{q}} \\
$$

Then we can compute the pUCB scores for the 2 actions in the initial round $N_s$:
$$
pUCB(p) = 1 + 0 - \frac{2(\sqrt{p}+\sqrt{q})}{\sqrt{p}} \\
pUCB(q) = 1 + 0 - \frac{2(\sqrt{p}+\sqrt{q})}{\sqrt{q}}
$$
where the default Q value is 1 because that is the maximum payoff/reward that can be achieved in the task. 

Since $p>q$ for assumption, action $p$ will be selected.

Now for $n \ge 2$, assuming that all previous rounds the action $p$ is selected and the reward $Q_p$ has been received, what will be the minimum $n$ for which action $q$ will be selected?

The new pUCB scores are:
$$
pUCB(p) = Q_p + \sqrt{\frac{3 \log n}{2 (n-1)}} - \frac{2(\sqrt{p}+\sqrt{q})}{\sqrt{p}}\sqrt{\frac{\log n}{n}} \\
pUCB(q) = 1 + 0 - \frac{2(\sqrt{p}+\sqrt{q})}{\sqrt{q}}\sqrt{\frac{\log n}{n}}
$$

Action $q$ will be selected only if $pUCB(p) < pUCB(q)$. Rearranging the terms we get:

$$
\Delta Q^* \ge \sqrt{\frac{\log n}{n}} \left[ \sqrt{\frac{3}{2}} + 2 \frac{2p-1}{\sqrt{p(1-p)}}\right]
$$
where I approximated $n-1 \approx n$ for simplicity and defined $\Delta Q^* \equiv 1 - Q_p$.

Without doing any computation we can see that $\forall \Delta Q^* > 0$ it exists a $n > \bar{n}$ for which this inequality is satisfied, since $\underset{n\rightarrow \infty}{lim} \sqrt{\frac{\log n}{n}} = 0$.

Calling 
$$
C_p = \sqrt{\frac{3}{2}} + 2 \frac{2p-1}{\sqrt{p(1-p)}} 
$$
$\bar{n}$ has to satisfy:


## Development - confront the 3 functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time
import copy
# custom imports
import utils
import train
#import mcts
from rtfm import featurizer as X
import os

Using device cpu
Using device cpu


In [2]:
from mcts import *

In [3]:
# We can confront the AlphaGo pUCT formula with the one I implemented with just minimal changes 
# to the original class
class PV_MCTS_debug(PolicyValueMCTS):
    def __init__(self, 
             root_frame,
             simulator,
             valid_actions,
             ucb_c,
             discount,
             max_actions,
             pv_net,
             root=None,
             render=False,
             ucb_method="p-UCT-old"
                ):

        super().__init__(
            root_frame,
            simulator,
            valid_actions,
            ucb_c,
            discount,
            max_actions,
            pv_net,
            root,
            render
        )
        
        possible_ucb_methods = ["p-UCT-old", "p-UCT-AlphaGo"]
        assert ucb_method in possible_ucb_methods, \
            ("ucb method not recognized, should be one of: ", possible_ucb_methods)
        self.ucb_method = ucb_method
        
    def ucb_score(self, parent, child):
        if self.ucb_method == "p-UCT-old":
            return self.ucb_score_old(parent, child)
        else:
            return self.ucb_score_AlphaGo(parent, child)
        
    def ucb_score_old(self, parent, child):
        """
        The score for a node is based on its value, plus an exploration bonus.
        """
        exploration_term = self.ucb_c*child.prior*np.sqrt(np.log(parent.visit_count)/(child.visit_count+1))

        if child.visit_count > 0:
            # Mean value Q
            value_term = child.reward + self.discount*child.value() 
        else:
            value_term = 0

        return value_term + exploration_term, value_term, exploration_term
    
    def ucb_score_AlphaGo(self, parent, child):
        """
        The score for a node is based on its value, plus an exploration bonus.
        """
        exploration_term = self.ucb_c*child.prior/(child.visit_count+1)*np.sqrt(parent.visit_count)

        if child.visit_count > 0:
            # Mean value Q
            value_term = child.reward + self.discount*child.value() 
        else:
            value_term = 0

        return value_term + exploration_term, value_term, exploration_term

In [4]:
# To implement the Rosen p-UCT formula instead we need to modify the node class too, in order 
# to store the weights M obtained from the probabilities and the expand function

class RosinPVNode(PriorValueNode):
    def __init__(self, prior=0., weight=0.):
        super().__init__(prior)
        # this weight is sqrt(prior)/sum_{all_children}sqrt(prior), needs to be computed beforehand
        self.weight = weight 
    
    def expand(self, frame, valid_actions, priors, reward, done, simulator):
        self.expanded = True
        vprint("Valid actions as child: ", valid_actions)
        vprint("Prior over the children: ", priors)
        weights = np.sqrt(priors)/np.sqrt(priors).sum()
        vprint("Weights over the children: ", weights)
        vprint("Terminal node: ", done)
        self.full_action_space = len(priors) # trick to pass this information
        self.frame = frame
        self.reward = reward
        self.terminal = done
        self.valid_actions = valid_actions
        if not done:
            for action in valid_actions:
                self.children[action] = RosinPVNode(priors[action], weights[action])
        self.simulator_dict = simulator.save_state_dict()
        
    def add_exploration_noise(self, dirichlet_alpha=0.5, exploration_fraction=0.25):
        """
        At the start of each search, we add dirichlet noise to the prior of the root to
        encourage the search to explore new actions.
        """
        actions = list(self.children.keys())
        noise = np.random.dirichlet([dirichlet_alpha] * len(actions))
        frac = exploration_fraction
        priors = []
        for a, n in zip(actions, noise):
            self.children[a].prior = self.children[a].prior * (1 - frac) + n * frac
            priors.append(self.children[a].prior) 
        # recompute all weights as the square root of the prior and then normalize them to 1
        priors = np.array(priors)
        weights = np.sqrt(priors)/np.sqrt(priors).sum()
        for i,a in enumerate(actions):
            self.children[a].weight = weights[i]

In [5]:
class Rosin_PV_MCTS(PolicyValueMCTS):
    def __init__(self, 
             root_frame,
             simulator,
             valid_actions,
             ucb_c,
             discount,
             max_actions,
             pv_net,
             root=None,
             render=False
                ):

        super().__init__(
            root_frame,
            simulator,
            valid_actions,
            ucb_c,
            discount,
            max_actions,
            pv_net,
            root,
            render
        )
        
    def run(self, num_simulations, mode="simulate", dir_noise=False, dirichlet_alpha=1.0, exploration_fraction=0.25):
        """
        Runs num_simulations searches starting from the root node corresponding to the internal
        state of the simulator given during initialization.
        Returns the root node and an extra_info dictionary
        """
        if self.root is None or self.root.visit_count==0:
            self.root = RosinPVNode() 
            
            with torch.no_grad():
                _, root_prior = self.pv_net(self.root_frame)
                root_prior = root_prior.reshape(-1).cpu().numpy()
                
                
            self.root.expand(
                self.root_frame,
                self.valid_actions,
                root_prior,
                0, # reward to get to root
                False, # terminal node
                self.simulator # state of the simulator at the root node 
            )
                
            # not sure about this
            self.root.visit_count += 1
            
        if dir_noise:
            # add noise to root even if the tree was inherited from previous time-step 
            self.root.add_exploration_noise(dirichlet_alpha, exploration_fraction)
                
        max_tree_depth = 0
        root = self.root
        #print("root.simulator_dict :", root.simulator_dict) # not ok
        #print("root: ", root) # ok
        #print("self.simulator: ", self.simulator)
        for n in range(num_simulations):
            ### Start of a simulation/search ###
            vprint("\nSimulation %d started."%(n+1))
            node = root
            # make sure that the simulator internal state is reset to the original one
            self.simulator.load_state_dict(root.simulator_dict)
            search_path = [node]
            current_tree_depth = 0
            if self.render:
                node.render(self.simulator)
            ### Selection phase until leaf node is reached ###
            while node.expanded or (current_tree_depth<self.max_actions):
                current_tree_depth += 1
                action, node = self.select(node)
                if self.render and node.expanded:
                    node.render(self.simulator)
                vprint("Current tree depth: ", current_tree_depth)
                vprint("Action selected: ", action, action_dict[action])
                vprint("Child node terminal: ", node.terminal)
                vprint("Child node expanded: ", node.expanded)
                if node.expanded or node.terminal:
                    search_path.append(node)
                    if node.terminal:
                        break
                else:
                    break
                
            ### Expansion of leaf node (if not terminal)###
            vprint("Expansion phase started")
            if not node.terminal:
                parent = search_path[-1] # last expanded node on the search path
                node = self.expand(node, parent, action)
                if self.render:
                    node.render(self.simulator)
                search_path.append(node)
            
            ### Simulation phase for self.max_actions - current_tree_depth steps ###
            vprint("Value prediction/simulation phase started")
            if mode=="simulate":
                value = self.simulate(node, current_tree_depth)
            elif mode=="predict":
                value = self.predict(node)
            elif mode=="simulate_and_predict":
                value = self.simulate_and_predict(node, current_tree_depth)
            elif mode =="hybrid":
                value1 = self.simulate(node, current_tree_depth)
                value2 =self.predict(node)
                value = 0.5*value1 + 0.5*value2
            else:
                raise Exception("Mode "+mode+" not implemented.")
            vprint("Predicted/simulated value: ", value)
            
            ### Backpropagation of the leaf node value along the seach_path ###
            vprint("Backpropagation phase started")
            self.backprop(search_path, value)
        
            max_tree_depth = max(max_tree_depth, current_tree_depth)
            vprint("Simulation %d done."%(n+1))
        extra_info = {
            "max_tree_depth": max_tree_depth
        }
        # just a check to see if root works as a shallow copy of self.root
        assert root.visit_count == self.root.visit_count, "self.root not updated during search"
        
        # make sure that the simulator internal state is reset to the original one
        self.simulator.load_state_dict(root.simulator_dict)
        return root, extra_info
    
    def ucb_score(self, parent, child):
        """
        The score for a node is based on its value, plus an exploration bonus.
        """
        # c_term increases p-UCT with time (i.e. parent's visit counts) and decreases with child's visit count
        if child.visit_count > 0:
            c_term = np.sqrt(3*np.log(parent.visit_count)/(2*child.visit_count))
        else:
            c_term = 0
        
        # m_term is a penalty term; increases for small probabilities and decreases with parent's visit counts
        if parent.visit_count > 1:
            m_term = 2/child.weight*np.sqrt(np.log(parent.visit_count)/parent.visit_count)
        else:
            m_term = 2/child.weight
            
        exploration_term = c_term - m_term
        
        if child.visit_count > 0:
            # Mean value Q
            value_term = child.reward + self.discount*child.value() 
        else:
            value_term = 1 # max payoff

        return value_term + exploration_term, value_term, exploration_term

In [3]:
def play_episode(
    frame,
    valid_actions,
    ucb_method,
    pv_net,
    env,
    episode_length,
    ucb_C,
    discount,
    max_actions,
    num_simulations,
    object_ids,
    mode="predict",
    dir_noise=False,
    render = True,
    debug_render=False,
    single_step = False
):
    """
    Plays an episode with a policy and value MCTS. 
    Starts building the tree from the sub-tree of the root's child node that has been selected at the previous step.
    
    If mode='simulate', it's identical to a policy MCTS with MC rollout evaluations, if mode='predict', the value network 
    is used to estimate the value of the leaf nodes (instead of a MC rollout).
    
    Chooses the best action as the one with highest Q-value according to the MCTS step and actually it's not returning
    any signal on which to train the policy (probably I used this to test a policy trained in a supervised fashion to
    predict the optimal actions given by a hard-coded policy; value net is not trained, thus this shoudl be used only
    in 'simulate' mode.
    """
    action_dict = {
        0:"Stay",
        1:"Up",
        2:"Down",
        3:"Left",
        4:"Right"
    }
    #frame, valid_actions = env.reset()
    if render:
        env.render()
    total_reward = 0
    done = False
    new_root = None
    # variables used for training of value net
    frame_lst = [frame]
    reward_lst = []
    done_lst = []
    action_is_optimal = []
    if render:
        prior_is_optimal = []
    for i in range(episode_length):
        
        tree = PV_MCTS(
                     frame, 
                     env, 
                     valid_actions, 
                     ucb_C, 
                     discount, 
                     max_actions, 
                     pv_net,
                     render=debug_render, 
                     root=new_root,
                     ucb_method=ucb_method
                     )

        #print("Performing MCTS step")
        root, info = tree.run(num_simulations, mode=mode, dir_noise=dir_noise)
        #show_root_summary(root, discount)
        #print("Tree info: ", info)
        action = root.best_action(discount)
        best_actions = utils.get_optimal_actions(frame, object_ids)
        if render:
            #print("probs from MCTS: ", probs)
            best_prior = show_policy_summary(pv_net, frame, root, discount, action, best_actions)
            
            if best_prior in best_actions:
                prior_is_optimal.append(True)
            else:
                prior_is_optimal.append(False)

        # Evaluate chosen action against optimal policy
        if action in best_actions:
            action_is_optimal.append(True)
        else:
            action_is_optimal.append(False)
            
        new_root = tree.get_subtree(action)
        frame, valid_actions, reward, done = env.step(action)
        
        frame_lst.append(frame)
        reward_lst.append(reward)
        done_lst.append(done)
        
        if render:
            env.render()
            print("Reward received: ", reward)
            print("Done: ", done)
        total_reward += reward
        if done:
            break
            
        if single_step:
            break
            
    if render:
        return total_reward, frame_lst, reward_lst, done_lst, action_is_optimal, prior_is_optimal
    else:
        return total_reward, frame_lst, reward_lst, done_lst, action_is_optimal

In [9]:
def compare_pUCT_formulas(
    pv_net, 
    game_simulator, 
    episode_length,
    ucb_C,
    discount,
    max_actions,
    num_simulations,
    object_ids,
    dir_noise=False,
    render=True,
    debug_render=False,
    single_step=False
):
    
    # Save original frame and simulator state dictionary to compare the 3 methods on the exact same episode
    # with the exact same network
    frame, valid_actions = game_simulator.reset()
    original_sim_state = game_simulator.save_state_dict()

    game_simulator.load_state_dict(copy.deepcopy(original_sim_state))
    print("-"*40)
    print("p-UCT old:")
    _ = play_episode(
            frame,
            valid_actions,
            "p-UCT-old",
            pv_net,
            game_simulator,
            episode_length,
            ucb_C,
            discount,
            max_actions,
            num_simulations,
            object_ids,
            dir_noise = dir_noise,
            render = render,
            debug_render = debug_render,
            single_step = single_step
    )
    
    game_simulator.load_state_dict(copy.deepcopy(original_sim_state))
    print("-"*40)
    print("p-UCT AlphaGo:")
    _ = play_episode(
            frame,
            valid_actions,
            "p-UCT-AlphaGo",
            pv_net,
            game_simulator,
            episode_length,
            ucb_C,
            discount,
            max_actions,
            num_simulations,
            object_ids,
            dir_noise = dir_noise,
            render = render,
            debug_render = debug_render,
            single_step = single_step
    )
    
    game_simulator.load_state_dict(copy.deepcopy(original_sim_state))
    print("-"*40)
    print("p-UCT Rosin:")
    _ = play_episode(
            frame,
            valid_actions,
            "p-UCT-Rosin",
            pv_net,
            game_simulator,
            episode_length,
            ucb_C,
            discount,
            max_actions,
            num_simulations,
            object_ids,
            dir_noise = dir_noise,
            render = render,
            debug_render = debug_render,
            single_step = single_step
    )

## Test on untrained pv net

In [5]:
from play_functions import show_policy_summary

In [6]:
# Check only if main logic of the training loop works
ucb_C = 1.0
discount = 0.9 # try with smaller discount
episode_length = 32
max_actions = 20
num_simulations = 50
#device = mcts.device
n_episodes = 4000
memory_size = 1024
batch_size = 32
n_steps = 5
tau = 0.1 # new_trg_params = (1-tau)*old_trg_params + tau*value_net_params
dir_noise = False
dirichlet_alpha = 0.5 # no real reason to choose this value, except it's < 1
exploration_fraction = 0.25
temperature = 1.
full_cross_entropy = True
entropy_bonus = True
entropy_weight = 1e-2

In [7]:
flags = utils.Flags(env="rtfm:groups_simple_stationary-v0")
gym_env = utils.create_env(flags)
featurizer = X.Render()
game_simulator = FullTrueSimulator(gym_env, featurizer)
object_ids = utils.get_object_ids_dict(game_simulator)

pv_net = DiscreteSupportPVNet_v3(gym_env).to(device)

### Single step - no root noise

In [10]:
compare_pUCT_formulas(
    pv_net, 
    game_simulator, 
    episode_length,
    ucb_C,
    discount,
    max_actions,
    num_simulations,
    object_ids,
    render=True,
    debug_render=False,
    single_step=True,
    dir_noise=False
)

----------------------------------------
p-UCT old:

██████
█   n█
█@y  █
█?   █
█!   █
██████

Action  Stay : Prior=0.177 - Q-value=-0.443 - Visit counts=2
Action  Up : Prior=0.213 - Q-value=-0.193 - Visit counts=4
Action  Down : Prior=0.320 - Q-value=-1.000 - Visit counts=1
Action  Right : Prior=0.290 - Q-value=-0.040 - Visit counts=43
Action with best prior:  2 (Down)
Action selected from MCTS:  4 (Right)
Best actions:  [4] ['Right']

██████
█   n█
█ @  █
█?   █
█!   █
██████
blessed sword
Reward received:  0
Done:  False
----------------------------------------
p-UCT AlphaGo:

██████
█   n█
█@y  █
█?   █
█!   █
██████

Action  Stay : Prior=0.177 - Q-value=-0.443 - Visit counts=2
Action  Up : Prior=0.213 - Q-value=-0.093 - Visit counts=15
Action  Down : Prior=0.320 - Q-value=-1.000 - Visit counts=2
Action  Right : Prior=0.290 - Q-value=-0.037 - Visit counts=31
Action with best prior:  2 (Down)
Action selected from MCTS:  4 (Right)
Best actions:  [4] ['Right']

██████
█   n█
█ @  █
█

In [11]:
# Debug render on
compare_pUCT_formulas(
    pv_net, 
    game_simulator, 
    episode_length,
    ucb_C,
    discount,
    max_actions,
    num_simulations,
    object_ids,
    render=True,
    debug_render=True,
    single_step=True,
    dir_noise=False
)

----------------------------------------
p-UCT old:

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

Valid actions as child:  [0 1 2 3 4]
Prior over the children:  [0.14916302 0.19584553 0.1838727  0.16505615 0.30606255]
Terminal node:  False

Simulation 1 started.

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

actions:  [0 1 2 3 4]
value_terms:  [0 0 0 0 0]
exploration_terms:  [0. 0. 0. 0. 0.]
ucb_values:  [0. 0. 0. 0. 0.]
max_U:  0.0
mask:  [ True  True  True  True  True]
best_actions:  [0 1 2 3 4]
Current tree depth:  1
Action selected:  1 Up
Child node terminal:  False
Child node expanded:  False
Expansion phase started
valid_actions:  [0 1 2 3 4]
prior:  [0.13791168 0.19442321 0.1973946  0.1719308  0.29833978]
reward:  0
done:  False
Valid actions as child:  [0 1 2 3 4]
Prior over the children:  [0.13791168 0.19442321 0.1973946  0.1719308  0.29833978]
Terminal node:  False

██████
█ !  █
█n@  █
█  y █
█  ? █
██████

Value prediction/simulation phase started
Predicted/simulated value:  -0.

ucb_values:  [0.114819   0.16186794 0.16434178 0.05940156 0.24838416]
max_U:  0.2483841629820077
mask:  [False False False False  True]
best_actions:  [4]
Current tree depth:  2
Action selected:  4 Right
Child node terminal:  False
Child node expanded:  False
Expansion phase started
valid_actions:  [0 1 2 3 4]
prior:  [0.16371053 0.19814923 0.18433253 0.16769339 0.2861143 ]
reward:  0
done:  False
Valid actions as child:  [0 1 2 3 4]
Prior over the children:  [0.16371053 0.19814923 0.18433253 0.16769339 0.2861143 ]
Terminal node:  False

██████
█ !  █
█n @ █
█  y █
█  ? █
██████

Value prediction/simulation phase started
Predicted/simulated value:  -0.04655185714364052
Backpropagation phase started
Simulation 12 done.

Simulation 13 started.

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

actions:  [0 1 2 3 4]
value_terms:  [-0.03206886 -0.03687645 -0.04017869 -0.03958509 -0.25070772]
exploration_terms:  [0.16892175 0.15682781 0.17001844 0.15261966 0.21921213]
ucb_values:  [ 0.13685289  0.

Predicted/simulated value:  -0.03506828099489212
Backpropagation phase started
Simulation 18 done.

Simulation 19 started.

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

actions:  [0 1 2 3 4]
value_terms:  [-0.03314455 -0.20789826 -0.32678579 -0.03601072 -0.25070772]
exploration_terms:  [0.12797716 0.13719533 0.15775696 0.14161296 0.23486942]
ucb_values:  [ 0.09483261 -0.07070294 -0.16902883  0.10560224 -0.0158383 ]
max_U:  0.10560224232414564
mask:  [False False False  True False]
best_actions:  [3]

██████
█ !  █
█n   █
█@ y █
█  ? █
██████

Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 4]
value_terms:  [-0.04166852  0.          0.         -0.03206886]
exploration_terms:  [0.12948467 0.23235171 0.24388687 0.27491569]
ucb_values:  [0.08781615 0.23235171 0.24388687 0.24284683]
max_U:  0.24388686772634677
mask:  [False False  True False]
best_actions:  [2]
Current tree depth:  2
Action selected:  2 Down
Child node te


██████
█ !  █
█n   █
█  @ █
█  ? █
██████
blessed sword
Value prediction/simulation phase started
Predicted/simulated value:  -0.03506828099489212
Backpropagation phase started
Simulation 24 done.

Simulation 25 started.

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

actions:  [0 1 2 3 4]
value_terms:  [-0.03261668 -0.20789826 -0.32678579 -0.03346789 -0.25070772]
exploration_terms:  [0.10925408 0.14344656 0.16494508 0.10469811 0.24557113]
ucb_values:  [ 0.0766374  -0.0644517  -0.16184071  0.07123022 -0.00513659]
max_U:  0.07663739508590288
mask:  [ True False False False False]
best_actions:  [0]

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

Current tree depth:  1
Action selected:  0 Stay
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 3 4]
value_terms:  [ 0.         -0.03528887 -0.04328831  0.         -0.03349728]
exploration_terms:  [0.18923362 0.17568544 0.16494508 0.20939622 0.22417474]
ucb_values:  [0.18923362 0.14039657 0.12165678 0.20939622 0.19067746]
max_U:


██████
█ !  █
█n   █
█@ y █
█  ? █
██████

Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 4]
value_terms:  [-0.04166852 -0.04181497 -0.04118785 -0.03063153]
exploration_terms:  [0.18311897 0.23235171 0.24388687 0.22446772]
ucb_values:  [0.14145045 0.19053675 0.20269902 0.19383619]
max_U:  0.20269901696791343
mask:  [False False  True False]
best_actions:  [2]

██████
█ !  █
█n   █
█  y █
█@ ? █
██████

Current tree depth:  2
Action selected:  2 Down
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 4]
value_terms:  [0 0 0]
exploration_terms:  [0. 0. 0.]
ucb_values:  [0. 0. 0.]
max_U:  0.0
mask:  [ True  True  True]
best_actions:  [0 1 4]
Current tree depth:  3
Action selected:  4 Right
Child node terminal:  False
Child node expanded:  False
Expansion phase started
valid_actions:  [0 1 3 4]
prior:  [0.18398207 0.23482426 0.         0.2153004  0.3658933 ]
reward:  0
done:  False
Valid actions as

...

max_U:  -3.1420441856781873
mask:  [False  True False False]
best_actions:  [1]

██████
█ ! @█
█n   █
█    █
█  ? █
██████
blessed sword
Current tree depth:  4
Action selected:  1 Up
Child node terminal:  False
Child node expanded:  True
actions:  [0 2 3]
value_terms:  [ 1.         -0.03528316  1.        ]
exploration_terms:  [-3.66852596 -2.30546037 -3.62328861]
ucb_values:  [-2.66852596 -2.34074353 -2.62328861]
max_U:  -2.3407435319614303
mask:  [False  True False]
best_actions:  [2]

██████
█ !  █
█n  @█
█    █
█  ? █
██████
blessed sword
Current tree depth:  5
Action selected:  2 Down
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 3]
value_terms:  [1 1 1 1]
exploration_terms:  [-8.45322871 -7.80816011 -7.56542541 -8.23380172]
ucb_values:  [-7.45322871 -6.80816011 -6.56542541 -7.23380172]
max_U:  -6.5654254093470605
mask:  [False False  True False]
best_actions:  [2]
Current tree depth:  6
Action selected:  2 Down
Child node terminal:  False
Child node expan

Predicted/simulated value:  -0.04655185714364052
Backpropagation phase started
Simulation 30 done.

Simulation 31 started.

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

actions:  [0 1 2 3 4]
value_terms:  [ 1.         -0.03649794  1.          1.         -0.11388383]
exploration_terms:  [-3.82116938 -1.7299686  -3.44166241 -3.63254507 -2.23869675]
ucb_values:  [-2.82116938 -1.76646653 -2.44166241 -2.63254507 -2.35258058]
max_U:  -1.7664665344491806
mask:  [False  True False False False]
best_actions:  [1]

██████
█ !  █
█n@  █
█  y █
█  ? █
██████

Current tree depth:  1
Action selected:  1 Up
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 3 4]
value_terms:  [ 1.          1.          1.          1.         -0.04189667]
exploration_terms:  [-7.03038664 -5.92114461 -5.87640996 -6.29655291 -3.76029175]
ucb_values:  [-6.03038664 -4.92114461 -4.87640996 -5.29655291 -3.80218842]
max_U:  -3.8021884211211465
mask:  [False False False False  True]
best_actions:  [4]

█████

actions:  [0 1 2 3]
value_terms:  [ 1.          1.         -0.03960613  1.        ]
exploration_terms:  [-4.9764581  -4.59670299 -3.43413686 -4.84728033]
ucb_values:  [-3.9764581  -3.59670299 -3.47374299 -3.84728033]
max_U:  -3.473742985890685
mask:  [False False  True False]
best_actions:  [2]

██████
█ !  █
█n   █
█   @█
█  ? █
██████
blessed sword
Current tree depth:  4
Action selected:  2 Down
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 3]
value_terms:  [1 1 1 1]
exploration_terms:  [-8.43225599 -7.90004363 -7.56992702 -8.14832955]
ucb_values:  [-7.43225599 -6.90004363 -6.56992702 -7.14832955]
max_U:  -6.569927016817344
mask:  [False False  True False]
best_actions:  [2]
Current tree depth:  5
Action selected:  2 Down
Child node terminal:  False
Child node expanded:  False
Expansion phase started
valid_actions:  [0 1 3]
prior:  [0.29727235 0.39036798 0.         0.3123597  0.        ]
reward:  0
done:  False
Valid actions as child:  [0 1 3]
Prior over the

valid_actions:  [0 1 3]
prior:  [0.29727235 0.39036798 0.         0.3123597  0.        ]
reward:  0
done:  False
Valid actions as child:  [0 1 3]
Prior over the children:  [0.29727235 0.39036798 0.         0.3123597  0.        ]
Weights over the children:  [0.31535822 0.36137992 0.         0.3232618  0.        ]
Terminal node:  False

██████
█ !  █
█n   █
█    █
█  ?@█
██████
blessed sword
Value prediction/simulation phase started
Predicted/simulated value:  -0.044320493936538696
Backpropagation phase started
Simulation 41 done.

Simulation 42 started.

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

actions:  [0 1 2 3 4]
value_terms:  [ 1.         -0.02906986  1.          1.         -0.09868536]
exploration_terms:  [-3.42494506 -2.0940636  -3.08478989 -3.25587956 -1.98492199]
ucb_values:  [-2.42494506 -2.12313346 -2.08478989 -2.25587956 -2.08360735]
max_U:  -2.083607352727273
mask:  [False False False False  True]
best_actions:  [4]

██████
█ !  █
█n   █
█  @ █
█  ? █
██████
blessed sword


valid_actions:  [0 1 2 3]
prior:  [0.21834621 0.28340966 0.2746046  0.22363955 0.        ]
reward:  0
done:  False
Valid actions as child:  [0 1 2 3]
Prior over the children:  [0.21834621 0.28340966 0.2746046  0.22363955 0.        ]
Weights over the children:  [0.23403898 0.2666384  0.26246372 0.23685889 0.        ]
Terminal node:  False

██████
█ !  █
█n   █
█  y@█
█  ? █
██████

Value prediction/simulation phase started
Predicted/simulated value:  -0.04500729963183403
Backpropagation phase started
Simulation 48 done.

Simulation 49 started.

██████
█ !  █
█n   █
█ @y █
█  ? █
██████

actions:  [0 1 2 3 4]
value_terms:  [ 1.         -0.02912729 -0.46803757  1.         -0.09457893]
exploration_terms:  [-3.23560985 -1.96953956 -1.70618909 -3.0758905  -1.85613011]
ucb_values:  [-2.23560985 -1.99866685 -2.17422666 -2.0758905  -1.95070905]
max_U:  -1.9507090456571752
mask:  [False False False False  True]
best_actions:  [4]

██████
█ !  █
█n   █
█  @ █
█  ? █
██████
blessed sword
Current t

## Test on trained pv net

In [11]:
# If enabled, use a trained network
load = True
load_dir="./save_dir"
ID="BAA"
checkpoint=6000
if load:
    d = torch.load("%s/%s/training_dict_%s"%(load_dir, ID, checkpoint), map_location=torch.device('cpu'))
    pv_net = d["pv_net"]

In [12]:
compare_pUCT_formulas(
    pv_net, 
    game_simulator, 
    episode_length,
    ucb_C,
    discount,
    max_actions,
    num_simulations,
    object_ids)

----------------------------------------
p-UCT old:

██████
█    █
█   ?█
█  n!█
█@y  █
██████

Action  Stay : Prior=0.001 - Q-value=0.000 - Visit counts=0
Action  Up : Prior=0.002 - Q-value=0.000 - Visit counts=0
Action  Right : Prior=0.997 - Q-value=0.729 - Visit counts=50
Action with best prior:  4 (Right)
Action selected from MCTS:  4 (Right)
Best actions:  [4] ['Right']

██████
█    █
█   ?█
█  n!█
█ @  █
██████
blessed sword
Reward received:  0
Done:  False
Action  Stay : Prior=0.000 - Q-value=0.000 - Visit counts=0
Action  Up : Prior=0.000 - Q-value=0.000 - Visit counts=0
Action  Left : Prior=0.000 - Q-value=0.000 - Visit counts=0
Action  Right : Prior=1.000 - Q-value=0.810 - Visit counts=99
Action with best prior:  4 (Right)
Action selected from MCTS:  4 (Right)
Best actions:  [4] ['Right']

██████
█    █
█   ?█
█  n!█
█  @ █
██████
blessed sword
Reward received:  0
Done:  False
Action  Stay : Prior=0.000 - Q-value=0.000 - Visit counts=0
Action  Up : Prior=0.000 - Q-value=0.000

In [18]:
compare_pUCT_formulas(
    pv_net, 
    game_simulator, 
    episode_length,
    ucb_C,
    discount,
    max_actions,
    num_simulations,
    object_ids,
    render=True,
    debug_render=True,
    single_step=True,
    dir_noise=False
)

----------------------------------------
p-UCT old:

██████
█  @!█
█    █
█n  ?█
█y   █
██████

Valid actions as child:  [0 2 3 4]
Prior over the children:  [6.1928187e-02 0.0000000e+00 3.1541348e-02 9.0637797e-01 1.5249924e-04]
Terminal node:  False

Simulation 1 started.

██████
█  @!█
█    █
█n  ?█
█y   █
██████

actions:  [0 2 3 4]
value_terms:  [0 0 0 0]
exploration_terms:  [0. 0. 0. 0.]
ucb_values:  [0. 0. 0. 0.]
max_U:  0.0
mask:  [ True  True  True  True]
best_actions:  [0 2 3 4]
Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expanded:  False
Expansion phase started
valid_actions:  [0 2 3 4]
prior:  [0.3316533  0.         0.29976946 0.3617069  0.00687034]
reward:  0
done:  False
Valid actions as child:  [0 2 3 4]
Prior over the children:  [0.3316533  0.         0.29976946 0.3617069  0.00687034]
Terminal node:  False

██████
█ @ !█
█    █
█n  ?█
█y   █
██████

Value prediction/simulation phase started
Predicted/simulated value:  0.35472828


██████
█   !█
█    █
█@  ?█
█y   █
██████
gleaming sword
Value prediction/simulation phase started
Predicted/simulated value:  0.5686187744140625
Backpropagation phase started
Simulation 6 done.

Simulation 7 started.

██████
█  @!█
█    █
█n  ?█
█y   █
██████

actions:  [0 2 3 4]
value_terms:  [0.         0.         0.31973944 0.        ]
exploration_terms:  [8.63872709e-02 4.39988813e-02 4.77883145e-01 2.12730163e-04]
ucb_values:  [8.63872709e-02 4.39988813e-02 7.97622589e-01 2.12730163e-04]
max_U:  0.7976225885992247
mask:  [False False  True False]
best_actions:  [3]

██████
█ @ !█
█    █
█n  ?█
█y   █
██████

Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expanded:  True
actions:  [0 2 3 4]
value_terms:  [0.        0.        0.3553736 0.       ]
exploration_terms:  [0.44393989 0.40126127 0.19766102 0.00919641]
ucb_values:  [0.44393989 0.40126127 0.55303462 0.00919641]
max_U:  0.5530346240012164
mask:  [False False  True False]
best_actions:

Current tree depth:  6
Action selected:  4 Right
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 3 4]
value_terms:  [0 0 0 0]
exploration_terms:  [0. 0. 0. 0.]
ucb_values:  [0. 0. 0. 0.]
max_U:  0.0
mask:  [ True  True  True  True]
best_actions:  [0 1 3 4]
Current tree depth:  7
Action selected:  0 Stay
Child node terminal:  False
Child node expanded:  False
Expansion phase started
valid_actions:  [0 1 3 4]
prior:  [0.00149575 0.95729417 0.         0.01875474 0.02245528]
reward:  0
done:  False
Valid actions as child:  [0 1 3 4]
Prior over the children:  [0.00149575 0.95729417 0.         0.01875474 0.02245528]
Terminal node:  False

██████
█   !█
█    █
█   ?█
█ @  █
██████
blessed sword
Value prediction/simulation phase started
Predicted/simulated value:  0.6839844584465027
Backpropagation phase started
Simulation 9 done.

Simulation 10 started.

██████
█  @!█
█    █
█n  ?█
█y   █
██████

actions:  [0 2 3 4]
value_terms:  [0.         0.         0.33362674 0.     

valid_actions:  [0 1 2 4]
prior:  [9.8140437e-05 9.8735631e-01 2.6330741e-05 0.0000000e+00 1.2519167e-02]
reward:  0
done:  False
Valid actions as child:  [0 1 2 4]
Prior over the children:  [9.8140437e-05 9.8735631e-01 2.6330741e-05 0.0000000e+00 1.2519167e-02]
Terminal node:  False

██████
█   !█
█@   █
█   ?█
█    █
██████
blessed sword
Value prediction/simulation phase started
Predicted/simulated value:  0.7727738618850708
Backpropagation phase started
Simulation 12 done.

Simulation 13 started.

██████
█  @!█
█    █
█n  ?█
█y   █
██████

actions:  [0 2 3 4]
value_terms:  [0.         0.         0.34314919 0.        ]
exploration_terms:  [9.91808350e-02 5.05149166e-02 4.02603052e-01 2.44234537e-04]
ucb_values:  [9.91808350e-02 5.05149166e-02 7.45752244e-01 2.44234537e-04]
max_U:  0.7457522437608857
mask:  [False False  True False]
best_actions:  [3]

██████
█ @ !█
█    █
█n  ?█
█y   █
██████

Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expa


██████
█ @ !█
█    █
█   ?█
█    █
██████
blessed sword
Value prediction/simulation phase started
Predicted/simulated value:  0.8999980092048645
Backpropagation phase started
Simulation 15 done.

Simulation 16 started.

██████
█  @!█
█    █
█n  ?█
█y   █
██████

actions:  [0 2 3 4]
value_terms:  [0.         0.         0.34580412 0.        ]
exploration_terms:  [1.03117195e-01 5.25197886e-02 3.77304580e-01 2.53927892e-04]
ucb_values:  [1.03117195e-01 5.25197886e-02 7.23108701e-01 2.53927892e-04]
max_U:  0.7231087014008499
mask:  [False False  True False]
best_actions:  [3]

██████
█ @ !█
█    █
█n  ?█
█y   █
██████

Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expanded:  True
actions:  [0 2 3 4]
value_terms:  [0.         0.         0.38633384 0.        ]
exploration_terms:  [0.54577379 0.49330526 0.15368785 0.01130594]
ucb_values:  [0.54577379 0.49330526 0.54002169 0.01130594]
max_U:  0.5457737886762746
mask:  [ True False False False]
best_act

...


██████
█   !█
█@   █
█n  ?█
█y   █
██████

Current tree depth:  4
Action selected:  2 Down
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 4]
value_terms:  [1.        1.        0.4900567 1.       ]
exploration_terms:  [ -4.451364   -70.51416845  -0.59054539 -26.72824578]
ucb_values:  [ -3.451364   -69.51416845  -0.10048869 -25.72824578]
max_U:  -0.10048868933474248
mask:  [False False  True False]
best_actions:  [2]

██████
█   !█
█    █
█@  ?█
█y   █
██████
gleaming sword
Current tree depth:  5
Action selected:  2 Down
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 4]
value_terms:  [1.         1.         0.54249817 1.        ]
exploration_terms:  [ -4.42070057 -40.84066785  -0.66888597 -15.3106349 ]
ucb_values:  [ -3.42070057 -39.84066785  -0.1263878  -14.3106349 ]
max_U:  -0.12638780207624445
mask:  [False False  True False]
best_actions:  [2]

██████
█   !█
█    █
█   ?█
█@   █
██████
blessed sword
Current tree depth:  6
Action selec


██████
█   @█
█    █
█   ?█
█    █
██████
blessed sword
Current tree depth:  11
Action selected:  4 Right
Child node terminal:  True
Child node expanded:  True
Expansion phase started
Value prediction/simulation phase started
Predicted/simulated value:  0
Backpropagation phase started
Simulation 44 done.

Simulation 45 started.

██████
█  @!█
█    █
█n  ?█
█y   █
██████

actions:  [0 2 3 4]
value_terms:  [1.         1.         0.33799402 1.        ]
exploration_terms:  [ -3.25109036  -4.55546905  -0.48956395 -65.51479936]
ucb_values:  [ -2.25109036  -3.55546905  -0.15156993 -64.51479936]
max_U:  -0.15156992705502192
mask:  [False False  True False]
best_actions:  [3]

██████
█ @ !█
█    █
█n  ?█
█y   █
██████

Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expanded:  True
actions:  [0 2 3 4]
value_terms:  [0.35247532 1.         0.3914363  1.        ]
exploration_terms:  [ -1.26326095  -1.93653817  -1.29570962 -12.79176678]
ucb_values:  [ -0.9107

actions:  [0 1 2 4]
value_terms:  [1.         0.72829131 1.         1.        ]
exploration_terms:  [-105.62570587   -0.45333164 -203.92105387   -9.35203044]
ucb_values:  [-104.62570587    0.27495967 -202.92105387   -8.35203044]
max_U:  0.2749596691571341
mask:  [False  True False False]
best_actions:  [1]

██████
█@  !█
█    █
█   ?█
█    █
██████
blessed sword
Current tree depth:  9
Action selected:  1 Up
Child node terminal:  False
Child node expanded:  True
actions:  [0 2 4]
value_terms:  [1.        1.        0.8099998]
exploration_terms:  [ -83.67410135 -188.62670159   -0.35637596]
ucb_values:  [ -82.67410135 -187.62670159    0.45362384]
max_U:  0.4536238423526959
mask:  [False False  True]
best_actions:  [4]

██████
█ @ !█
█    █
█   ?█
█    █
██████
blessed sword
Current tree depth:  10
Action selected:  4 Right
Child node terminal:  False
Child node expanded:  True
actions:  [0 2 3 4]
value_terms:  [1.  1.  1.  0.9]
exploration_terms:  [ -53.31218259 -237.69814025 -288.39697272

value_terms:  [1.         1.         0.53674844 1.        ]
exploration_terms:  [ -3.52304896 -32.54770824  -0.54218159 -12.20171226]
ucb_values:  [-2.52304896e+00 -3.15477082e+01 -5.43314452e-03 -1.12017123e+01]
max_U:  -0.005433144523207423
mask:  [False False  True False]
best_actions:  [2]

██████
█   !█
█    █
█   ?█
█@   █
██████
blessed sword
Current tree depth:  5
Action selected:  2 Down
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 4]
value_terms:  [1.         0.59346598 1.        ]
exploration_terms:  [-5.74580095 -0.89891165 -2.09532964]
ucb_values:  [-4.74580095 -0.30544567 -1.09532964]
max_U:  -0.30544567237123055
mask:  [False  True False]
best_actions:  [1]

██████
█   !█
█    █
█@  ?█
█    █
██████
blessed sword
Current tree depth:  6
Action selected:  1 Up
Child node terminal:  False
Child node expanded:  True
actions:  [0 1 2 4]
value_terms:  [1.         0.65753558 1.         1.        ]
exploration_terms:  [-36.61859378  -0.43143005 -50.79433


██████
█   @█
█    █
█   ?█
█    █
██████
blessed sword
Current tree depth:  12
Action selected:  4 Right
Child node terminal:  True
Child node expanded:  True
Expansion phase started
Value prediction/simulation phase started
Predicted/simulated value:  0
Backpropagation phase started
Simulation 49 done.

Simulation 50 started.

██████
█  @!█
█    █
█n  ?█
█y   █
██████

actions:  [0 2 3 4]
value_terms:  [1.         1.         0.33766109 1.        ]
exploration_terms:  [ -3.12664669  -4.381097    -0.47121768 -63.00705543]
ucb_values:  [ -2.12664669  -3.381097    -0.13355659 -62.00705543]
max_U:  -0.1335565887615352
mask:  [False False  True False]
best_actions:  [3]

██████
█ @ !█
█    █
█n  ?█
█y   █
██████

Current tree depth:  1
Action selected:  3 Left
Child node terminal:  False
Child node expanded:  True
actions:  [0 2 3 4]
value_terms:  [0.35207564 1.         0.39102087 1.        ]
exploration_terms:  [ -1.21497663  -1.86099113  -1.24551423 -12.29274222]
ucb_values:  [ -0.86290